In [7]:

import os
import ROOT
import uproot
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
import matplotlib.colors as colors
plt.style.use('/home/llr/ilc/shi/code/Plot_style/cepc.mplstyle')
fig_size = (6, 6)
Cell_X_No=40
Cell_Y_No=40
Layer_No=80
Cell_X=5.0
Cell_Y=5.0
Si_Z=0.15
Cell_Z=2.25
Start_Z=1.575
def decode_volid_and_indices(volid_array):
    volid_array = volid_array.astype(np.int64)
    calolayer = volid_array & 0x7F
    abslayer  = (volid_array >> 7) & 0x1
    cellid    = (volid_array >> 8) & 0x1FFF
    index_z = cellid // 1600
    index_y = (cellid % 1600) // 40
    index_x = cellid % 40
    return calolayer, abslayer, index_x, index_y, index_z
def encode_volid_array(calolayer, abslayer, index_x, index_y, index_z):
    cellid = index_x + 40 * index_y + 1600 * index_z
    volid = (calolayer & 0x7F) | ((abslayer & 0x1) << 7) | (cellid << 8)
    return volid.astype(np.int64)

In [ ]:
def combine_cells(input_file, output_file, CombineFactor_X, CombineFactor_Y, CombineFactor_Si, CombineFactor_layer):
    #Set up input
    f_in = uproot.open(input_file)
    tree_in = f_in["events"]
    cellID = tree_in["simplecaloRO.cellID"].array(library="np")#int64
    energy = tree_in["simplecaloRO.energy"].array(library="np")#float32
    pos_x  = tree_in["simplecaloRO.position.x"].array(library="np")#float32
    pos_y  = tree_in["simplecaloRO.position.y"].array(library="np")#float32
    pos_z  = tree_in["simplecaloRO.position.z"].array(library="np")#float32

    #Set up output
    cellID_out=[]
    energy_out=[]
    pos_x_out=[]
    pos_y_out=[]
    pos_z_out=[]
    Combined_Cell_X = Cell_X*CombineFactor_X#mm
    Combined_Cell_Y = Cell_Y*CombineFactor_Y#mm
    Combined_Si = Si_Z*CombineFactor_Si#mm
    Combined_Z = Cell_Z*CombineFactor_layer#mm
    Combined_layerNo = int(Layer_No / CombineFactor_layer)
    Combined_X_No = int(Cell_X_No / CombineFactor_X)
    Combined_Y_No = int(Cell_Y_No / CombineFactor_Y)
    print("Combined Parameters")
    print("  Combined_X:", Combined_Cell_X, "mm", "Combined_X_No:", Combined_X_No)
    print("  Combined_Y:", Combined_Cell_Y, "mm", "Combined_Y_No:", Combined_Y_No)
    print("  Combined_Si:", Combined_Si, "mm")
    print("  Combined_Z:", Combined_Z, "mm", "Combined_layerNo:", Combined_layerNo)
    #Loop input and combine cells
    #for i_event in range(len(cellID)):
    for i_event in range(1):    
        if i_event % 10 == 0:
            print("Processing event:", i_event)
        calolayer, abslayer, index_x, index_y, index_z = decode_volid_and_indices(cellID[i_event])
        energy_i= energy[i_event]
        mask = pos_x[i_event] >10
        print("  before combining (pos_z == 12.5):")
        print("    calolayer:", calolayer[mask])
        print("    abslayer:", abslayer[mask])
        print("    index_x:", index_x[mask])
        print("    index_y:", index_y[mask])
        print("    index_z:", index_z[mask])
        print("    energy_i:", energy_i[mask])
        print("    pos_x:", pos_x[i_event][mask])
        print("    pos_y:", pos_y[i_event][mask])
        print("    pos_z:", pos_z[i_event][mask])
        #Select Si
        mask = index_z < CombineFactor_Si
        calolayer, abslayer, index_x, index_y, index_z, energy_i = calolayer[mask], abslayer[mask], index_x[mask], index_y[mask], index_z[mask], energy_i[mask]
        #Combine X,Y,layer
        Combined_index_x = index_x // CombineFactor_X
        Combined_index_y = index_y // CombineFactor_Y
        Combined_index_z = np.full_like(Combined_index_x, CombineFactor_Si-1, dtype=np.int64)
        Combined_calolayer = calolayer // CombineFactor_layer
        Combined_abslayer = abslayer
        Combined_cellID = encode_volid_array(Combined_calolayer, Combined_abslayer, Combined_index_x, Combined_index_y, Combined_index_z)
        Combined_calolayer, Combined_abslayer, Combined_index_x, Combined_index_y, Combined_index_z = decode_volid_and_indices(Combined_cellID)
        Combined_cellID, unique_indices = np.unique(Combined_cellID, return_inverse=True)
        Combined_energy = np.zeros_like(Combined_cellID, dtype=np.float32)
        np.add.at(Combined_energy, unique_indices, energy_i)
        Combined_calolayer, Combined_abslayer, Combined_index_x, Combined_index_y, Combined_index_z = decode_volid_and_indices(Combined_cellID)
        Combined_pos_x = (Combined_index_x - Combined_X_No / 2.0 + 0.5) * Combined_Cell_X
        Combined_pos_y = (Combined_index_y - Combined_Y_No / 2.0 + 0.5) * Combined_Cell_Y
        Combined_pos_z = Start_Z + (Combined_calolayer-1) * Combined_Z
        mask = Combined_pos_x > 10
        print("  after combining :")
        print("    calolayer:", Combined_calolayer[mask])
        print("    abslayer:", Combined_abslayer[mask])
        print("    index_x:", Combined_index_x[mask])
        print("    index_y:", Combined_index_y[mask])
        print("    index_z:", Combined_index_z[mask])
        print("    energy_i:", Combined_energy[mask])
        print("    pos_x:", Combined_pos_x[mask])
        print("    pos_y:", Combined_pos_y[mask])
        print("    pos_z:", Combined_pos_z[mask])
        #Store output
        cellID_out.append(Combined_cellID)
        energy_out.append(Combined_energy)
        pos_x_out.append(Combined_pos_x)
        pos_y_out.append(Combined_pos_y)
        pos_z_out.append(Combined_pos_z)
    f_in.close()
    print("Output arrays:")
    print("  energy_out:", np.concatenate(energy_out))
    print(cellID_out[:2])
    print(energy_out[:2])

    array_dict = {
        "simplecaloRO.cellID": ak.Array(cellID_out),         # list of np.int64 arrays
        "simplecaloRO.energy": ak.Array(energy_out),         # list of np.float32 arrays
        "simplecaloRO.position.x": ak.Array(pos_x_out),      # ...
        "simplecaloRO.position.y": ak.Array(pos_y_out),
        "simplecaloRO.position.z": ak.Array(pos_z_out),
    }
    with uproot.recreate(output_file) as f_out:
        f_out["events"] = array_dict

   
        # f_out["events"] = uproot.newtree({
            # "simplecaloRO.cellID": "int64",
            # "simplecaloRO.energy": "float32",
            # "simplecaloRO.position.x": "float32",
            # "simplecaloRO.position.y": "float32",
            # "simplecaloRO.position.z": "float32",
        # })
        # f_out["events"].extend({
            # "simplecaloRO.cellID": cellID_out,
            # "simplecaloRO.energy": energy_out,
            # "simplecaloRO.position.x": pos_x_out,
            # "simplecaloRO.position.y": pos_y_out,
            # "simplecaloRO.position.z": pos_z_out,
        # })

In [9]:
def main():

    CombineFactor_X = 2
    CombineFactor_Y = 2
    CombineFactor_Si = 5
    CombineFactor_layer=1
    Combined_X = Cell_X*CombineFactor_X#mm
    Combined_Y = Cell_Y*CombineFactor_Y#mm
    Combined_Si = Si_Z*CombineFactor_Si#mm
    Combined_Z = Cell_Z*CombineFactor_layer#mm
    Combined_layerNo = int(Layer_No / CombineFactor_layer)
    Energy = 100.0#GeV
    data_path = "/data_ilc/flc/shi/SiWECAL-Prototype/Simu2025-06/CONF0/mu-"
    input_file = f"{data_path}/MC/{Energy}GeV.root"
    output_file = f"{data_path}/Merged_X{Combined_X}mm_Y{Combined_Y}mm_Si{Combined_Si}mm_layer{Combined_layerNo}/{Energy}GeV.root"
    output_dir = os.path.dirname(output_file)
    os.makedirs(output_dir, exist_ok=True)
    combine_cells(input_file, output_file, CombineFactor_X, CombineFactor_Y, CombineFactor_Si, CombineFactor_layer)


if __name__ == "__main__":
    main()


Combined Parameters
  Combined_X: 10.0 mm Combined_X_No: 20
  Combined_Y: 10.0 mm Combined_Y_No: 20
  Combined_Si: 0.75 mm
  Combined_Z: 2.25 mm Combined_layerNo: 80
Processing event: 0
  before combining (pos_z == 12.5):
    calolayer: [19 19 19 19 19 19 19 19]
    abslayer: [0 0 0 0 0 0 0 0]
    index_x: [21 21 21 21 21 22 22 22]
    index_y: [20 20 19 19 20 20 20 20]
    index_z: [4 3 3 2 2 2 1 0]
    energy_i: [5.3400814e-04 1.8338140e-05 8.2120379e-05 2.0620361e-04 2.4267647e-05
 1.7225491e-04 1.1792597e-04 1.9119741e-04]
    pos_x: [ 7.5  7.5  7.5  7.5  7.5 12.5 12.5 12.5]
    pos_y: [ 2.5  2.5 -2.5 -2.5  2.5  2.5  2.5  2.5]
    pos_z: [42.675 42.525 42.525 42.375 42.375 42.375 42.225 42.075]
  after combining :
    calolayer: [10 19  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46
 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70
 71 72 73 74 75 76 77 78 79 80 19]
   